# Parameters List
1. k_batch=list(np.arange(1.0, 4.0, 0.5)) # k parameter

2. k_instance=list(np.arange(1.0, 4.0, 0.5)) # This can be set the same as k_batch and hence excluded

3. count_threshold_measure=['median','max'] # alpha parameter

4. logical_parameter=['and','or']



In [1]:
import numpy as np
k_batch=list(np.arange(1.0, 4.5, 0.5))
count_threshold_measure=['median','max']
logical_parameter=['and','or']

In [2]:
import itertools

hyperparameter_space = {
    'k_batch': k_batch,
    'count_threshold_measure':count_threshold_measure,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))


In [3]:
all_combinations

[(1.0, 'median', 'and'),
 (1.0, 'median', 'or'),
 (1.0, 'max', 'and'),
 (1.0, 'max', 'or'),
 (1.5, 'median', 'and'),
 (1.5, 'median', 'or'),
 (1.5, 'max', 'and'),
 (1.5, 'max', 'or'),
 (2.0, 'median', 'and'),
 (2.0, 'median', 'or'),
 (2.0, 'max', 'and'),
 (2.0, 'max', 'or'),
 (2.5, 'median', 'and'),
 (2.5, 'median', 'or'),
 (2.5, 'max', 'and'),
 (2.5, 'max', 'or'),
 (3.0, 'median', 'and'),
 (3.0, 'median', 'or'),
 (3.0, 'max', 'and'),
 (3.0, 'max', 'or'),
 (3.5, 'median', 'and'),
 (3.5, 'median', 'or'),
 (3.5, 'max', 'and'),
 (3.5, 'max', 'or'),
 (4.0, 'median', 'and'),
 (4.0, 'median', 'or'),
 (4.0, 'max', 'and'),
 (4.0, 'max', 'or')]

In [4]:
len(all_combinations)

28

### Design the Process flow for grid search

1. Autoencoder is pre-trained , which will be used for all combinations ,this will reduce the processing overhead
2. The validation data needs to be processed once 
3. The iterations will start from threshold computations 
4. The objective is least warnings and detections on normal data set , least detection delay , and more warnnings after detection point
5. We can set the normal data size 20 batches , drifted data size also 20 batches , both combined with drift starting from batch 20
6. We can assign weights to factors in (5) and define a score



In [5]:
total_batches=40
batch_size=32
drift_point=20

WnDs=[i for i in range(0,21)]
print("Warnings in Drift Stream before drift point",WnDs)
DnDs=[i for i in range(0,21)]
print("Detections in Drift Stream before drift point",DnDs)
delay=[ i for i in range(21)]
print("Detection Delay:",delay)
DdDs=[i for i in range(19)]
print("Detections after drift point:",DdDs)
WdDs=[i for i in range(19)]
print("Warnings after drift point:",WdDs)

Warnings in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detection Delay: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Warnings after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [6]:
WnDS=[i for i in range(0,21)]
DnDs=[i for i in range(0,21)]
delay=[ i for i in range(21)]
DdDs=[i for i in range(19)]
WdDs=[i for i in range(19)]

In [7]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    best_value = float('-inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value > best_value:
                            best_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, best_value

# Find the best parameters and the corresponding value
best_parameters, best_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Best parameters:', best_parameters)
print('Value:', best_value)

Best parameters: (0, 0, 0, 18, 18)
Value: 54


In [8]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    worst_value = float('inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value < worst_value:
                            worst_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, worst_value

# Find the best parameters and the corresponding value
best_parameters, worst_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Worst parameters:', best_parameters)
print('Value:', worst_value)

Worst parameters: (20, 20, 20, 0, 0)
Value: -120


# Dataset 

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing


from skmultiflow.data import HyperplaneGenerator
from scipy.stats import norm
import random
from scipy import stats 


import tensorflow
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import anderson

In [10]:
data=pd.read_csv('rbf_nondrifted.csv')

In [11]:
data

,att0,att1,att2,att3,att4,att5,att6,att7,att8,att9,...,att21,att22,att23,att24,att25,att26,att27,att28,att29,output
0,0.627878,0.090092,0.292179,0.426788,0.691710,0.863260,0.586269,0.434148,0.348372,0.515820,...,0.514707,0.421725,0.736112,0.249198,0.289654,0.043979,-0.041867,1.000854,0.524041,1
1,1.087111,0.873756,0.297654,0.374160,0.736810,0.758003,0.115706,0.582408,1.050392,0.765421,...,0.614297,-0.070310,0.604598,0.561203,-0.058381,0.116014,0.042062,0.144130,0.190421,0
2,-0.000112,0.848397,0.798158,0.215230,0.662737,0.846810,-0.122763,0.494109,0.232414,0.720528,...,0.232952,0.558717,0.689361,0.074247,0.826573,0.572415,0.568676,-0.014577,0.708819,1
3,1.489162,-0.160488,-0.399419,0.437733,0.241093,0.896487,-0.295125,0.415727,1.277070,0.120752,...,0.935642,0.526834,-0.173683,0.299481,-0.373176,-0.314458,-0.387125,0.187890,0.904175,1
4,0.719700,0.720847,0.737085,-0.052983,0.568074,0.619193,0.583476,0.248259,0.819285,0.613443,...,0.607549,0.226472,1.006646,0.777454,-0.042836,-0.027309,0.741685,-0.079716,0.252760,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0.228406,-0.297779,0.570945,0.743156,0.813900,0.493693,0.768333,-0.340724,0.291664,-0.079476,...,0.356663,0.785386,-0.334328,-0.120836,0.425670,-0.129934,0.816731,0.771300,0.692432,1
39996,0.571247,0.669654,0.774814,0.381969,0.168584,0.196572,0.161200,0.319425,0.407750,0.450691,...,0.153179,0.027313,0.103856,0.907571,0.598629,0.216149,0.563697,0.169167,0.694615,0
39997,0.669387,0.490296,0.830206,0.032757,0.805616,0.569088,0.296901,0.042715,0.990293,0.006849,...,0.552440,0.290681,0.819033,0.830857,0.219658,0.649324,0.091910,0.411600,0.100830,1
39998,0.337685,0.711524,0.651867,0.099410,0.700417,0.068234,0.204345,0.074575,0.213803,0.975817,...,1.021073,0.512862,0.867969,0.717582,0.749886,0.263585,0.534637,0.786471,0.577160,1


In [12]:
drifted_data=pd.read_csv('rbf_drifted.csv')

In [13]:
drifted_data

,att0,att1,att2,att3,att4,att5,att6,att7,att8,att9,...,att21,att22,att23,att24,att25,att26,att27,att28,att29,output
0,-0.095989,0.873258,0.852772,0.434076,0.429089,0.939601,0.037137,0.493261,0.369151,0.795706,...,0.454775,0.693296,0.536494,0.308500,0.754002,0.639403,0.401558,-0.020486,0.825145,1
1,-0.180851,0.500238,0.472646,0.134438,0.276180,0.672549,0.105144,0.190629,-0.212791,0.280026,...,0.575546,0.419622,0.804660,0.415955,0.842476,-0.076269,0.428833,0.051230,0.914816,0
2,0.291043,0.870622,0.317557,0.132319,0.861191,0.794154,0.671193,0.811855,0.679301,0.899769,...,0.252147,0.790719,0.799345,0.462623,0.745273,0.269648,0.668613,0.803265,0.653427,1
3,0.545315,0.307460,0.517104,0.528226,0.358875,0.366911,0.304618,0.143410,0.856648,0.547584,...,0.154688,0.333292,0.032872,0.237570,0.608880,0.703455,0.225914,0.194997,0.292264,0
4,0.917142,0.431144,0.421769,0.669505,0.697606,0.047366,0.518574,0.736057,0.094281,0.048495,...,0.829549,0.717949,0.722405,0.292935,0.367568,0.901941,0.494181,0.560104,0.593034,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,1.005338,0.403683,0.949923,0.509371,0.298579,0.460136,0.197672,0.679426,0.434426,0.656409,...,0.750499,0.235856,0.250975,0.297550,0.244202,0.079738,0.553779,0.050856,0.247205,0
2556,0.563452,0.159565,0.684248,0.083377,0.430657,0.104970,0.650416,0.161691,0.380668,0.793702,...,0.335834,0.404869,0.034894,0.538874,0.275189,0.774527,0.002005,0.035961,0.464705,0
2557,0.650439,-0.107468,1.045493,0.624431,0.428316,0.310440,-0.009045,0.461736,0.510145,0.609477,...,0.677693,0.353645,0.156902,0.751247,-0.055700,-0.066099,0.904765,-0.012248,-0.125438,0
2558,0.547455,0.870983,0.893553,0.259121,0.405676,1.030035,-0.151114,0.309036,0.682503,0.271462,...,0.774388,-0.136439,0.312725,0.375155,0.485545,0.365546,-0.144133,0.312322,0.400043,1


In [14]:
del drifted_data['output']

In [15]:
drifted_data=drifted_data[0:1280]

## 3. Data Preprocessing 
1. Initial 70% data is used for training ( training and validation for AE)
2. Next 20 % data is used as a validation set to compute thresholds
3. Next 10 % data is used as a test stream initilally without drift 
  

In [16]:
def normalize_encode_split(data,label_col,pos_val,neg_val):        
    # Apply Label Encoding 
    # Apply MinMAx Normalization
    # Split into training  , test (validation set 1) and stream ( non-drifted data stream)   
            
    train=data[0:int(len(data)*0.70)]
    test=data[int((0.70*len(data))):int((0.90*len(data)))]
    stream=data[int((0.90*len(data))):len(data)]
    
    train_positives = train[train[label_col] == pos_val]
    train_negatives = train[train[label_col] == neg_val]
    
    
    y_positive=train_positives[label_col] 
    y_negative=train_negatives[label_col]
    X_positive=train_positives.drop([label_col],axis=1)
    X_negative=train_negatives.drop([label_col],axis=1)
    
    
    return train, test, X_positive,X_negative , stream ,y_positive ,y_negative  

In [17]:
train, test, X_positive,X_negative , stream, y_positive ,y_negative =normalize_encode_split(data,'output',1,0)

### Best Autoencoder Hyperparameters will be deternmined for each data set separetely 

In [18]:
def autoencoder (train,epochs,val_set):
    
    # set bottleneck 1/3 of the input layer size
    #encoding_dim = int(np.floor(train.shape[-1]/3))
       
    # input layer size= # of attributes in the dataset after one-hot encoding
    input_layer = tensorflow.keras.Input(shape=train.shape[1:]) # Input Layer
    encoded = layers.Dense(6, activation='relu')(input_layer) # Code Layer 1
    encoded=layers.Dense(2,activation='relu')(encoded) # Bottleneck
    decoded=layers.Dense(6,activation='relu')(encoded)# Decode Layer 1
    decoded = layers.Dense(train.shape[1], activation='sigmoid')(decoded) # Output Layer
          
    autoencoder = tensorflow.keras.Model(input_layer, decoded)
    # above model maps an input to its reconstruction 
   
    autoencoder.compile(optimizer='adam', loss='mse') # Train autoencoder 
    history=autoencoder.fit(train,train, 
                epochs=epochs,
                batch_size=32,
                shuffle=True,
                validation_data=(val_set, val_set)).history
    return  autoencoder , history

In [19]:
def train_encoders(X_Positive,X_Negative, epochs):
    
    X_Positive_train=X_Positive[0:int(len(X_Positive)*0.90)]
    X_Positive_test=X_Positive[int((0.90*len(X_Positive))):len(X_Positive)-1]
    
    X_Negative_train=X_Negative[0:int(len(X_Negative)*0.90)]
    X_Negative_test=X_Negative[int((0.90*len(X_Negative))):len(X_Negative)-1]

    print("Training Autoencoder on Positive Examples ")
    encoder_pos_class, history_positive_class=autoencoder(X_Positive_train,epochs,X_Positive_test)
    print("Training Autoencoder on Negative Examples ")
    encoder_neg_class,history_negative_class=autoencoder(X_Negative_train,epochs,X_Negative_test)
    
    return encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class

In [20]:
encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class=train_encoders(X_positive,X_negative, 100)

Training Autoencoder on Positive Examples 
Epoch 1/100
432/432 [==============================] - 3s 6ms/step - loss: 0.0943 - val_loss: 0.0856
Epoch 2/100
432/432 [==============================] - 1s 3ms/step - loss: 0.0844 - val_loss: 0.0785
Epoch 3/100
432/432 [==============================] - 1s 2ms/step - loss: 0.0764 - val_loss: 0.0707
Epoch 4/100
432/432 [==============================] - 1s 3ms/step - loss: 0.0698 - val_loss: 0.0672
Epoch 5/100
432/432 [==============================] - 1s 3ms/step - loss: 0.0672 - val_loss: 0.0658
Epoch 6/100
432/432 [==============================] - 1s 3ms/step - loss: 0.0654 - val_loss: 0.0647
Epoch 7/100
432/432 [==============================] - 1s 3ms/step - loss: 0.0647 - val_loss: 0.0637
Epoch 8/100
432/432 [==============================] - 1s 3ms/step - loss: 0.0638 - val_loss: 0.0628
Epoch 9/100
432/432 [==============================] - 1s 3ms/step - loss: 0.0630 - val_loss: 0.0620
Epoch 10/100
432/432 [==========================

Epoch 81/100
432/432 [==============================] - 3s 7ms/step - loss: 0.0522 - val_loss: 0.0518
Epoch 82/100
432/432 [==============================] - 3s 7ms/step - loss: 0.0524 - val_loss: 0.0519
Epoch 83/100
432/432 [==============================] - 3s 7ms/step - loss: 0.0516 - val_loss: 0.0520
Epoch 84/100
432/432 [==============================] - 5s 12ms/step - loss: 0.0522 - val_loss: 0.0518
Epoch 85/100
432/432 [==============================] - 3s 7ms/step - loss: 0.0515 - val_loss: 0.0518
Epoch 86/100
432/432 [==============================] - 3s 7ms/step - loss: 0.0515 - val_loss: 0.0518
Epoch 87/100
432/432 [==============================] - 3s 7ms/step - loss: 0.0520 - val_loss: 0.0518
Epoch 88/100
432/432 [==============================] - 3s 7ms/step - loss: 0.0520 - val_loss: 0.0517
Epoch 89/100
432/432 [==============================] - 3s 7ms/step - loss: 0.0517 - val_loss: 0.0518
Epoch 90/100
432/432 [==============================] - 3s 7ms/step - loss: 0.051

Epoch 61/100
356/356 [==============================] - 3s 8ms/step - loss: 0.0553 - val_loss: 0.0570
Epoch 62/100
356/356 [==============================] - 3s 8ms/step - loss: 0.0549 - val_loss: 0.0569
Epoch 63/100
356/356 [==============================] - 3s 8ms/step - loss: 0.0548 - val_loss: 0.0569
Epoch 64/100
356/356 [==============================] - 3s 8ms/step - loss: 0.0546 - val_loss: 0.0568
Epoch 65/100
356/356 [==============================] - 3s 8ms/step - loss: 0.0549 - val_loss: 0.0569
Epoch 66/100
356/356 [==============================] - 3s 8ms/step - loss: 0.0546 - val_loss: 0.0569
Epoch 67/100
356/356 [==============================] - 1s 3ms/step - loss: 0.0551 - val_loss: 0.0569
Epoch 68/100
356/356 [==============================] - 3s 8ms/step - loss: 0.0545 - val_loss: 0.0568
Epoch 69/100
356/356 [==============================] - 3s 9ms/step - loss: 0.0546 - val_loss: 0.0568
Epoch 70/100
356/356 [==============================] - 3s 8ms/step - loss: 0.0552

In [21]:
test

,att0,att1,att2,att3,att4,att5,att6,att7,att8,att9,...,att21,att22,att23,att24,att25,att26,att27,att28,att29,output
28000,0.325504,0.153972,0.450322,0.786900,0.871411,0.578598,0.267564,0.475857,1.017090,0.858609,...,0.649676,0.569793,0.514537,0.604950,0.901414,0.153487,0.906252,0.257675,-0.060496,0
28001,0.965002,0.817406,0.296818,0.956844,0.688823,0.400714,0.962603,0.264181,0.763586,0.633173,...,0.114791,0.747484,0.146099,0.871947,0.683196,0.958914,0.063107,0.248996,0.541730,0
28002,0.605548,0.357328,0.093962,0.949161,0.936269,0.484559,0.302181,0.501106,0.240436,0.045960,...,1.076587,0.152237,0.501185,0.654182,0.814172,0.375639,0.008005,0.781714,0.619034,0
28003,0.291502,0.868071,0.321986,0.130093,0.854430,0.788509,0.670362,0.808345,0.676239,0.905058,...,0.260005,0.791025,0.791201,0.467387,0.744806,0.277959,0.661235,0.796604,0.646184,1
28004,-0.029797,0.923406,-0.029181,0.600163,0.278502,0.504161,0.777265,0.536354,-0.038002,0.343635,...,0.121797,0.263772,0.293542,0.389014,0.639456,0.603824,0.747750,0.498396,0.545014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,0.618921,0.667181,0.673525,0.527658,0.325354,1.033250,0.600980,0.961296,0.803172,0.185021,...,0.363512,0.853430,-0.053858,-0.027543,-0.062115,0.424981,0.713808,0.360889,0.487185,0
35996,0.354305,0.694507,0.641685,0.062175,0.666898,0.055667,0.231559,0.111154,0.236568,0.953918,...,0.998616,0.533626,0.830288,0.724883,0.773718,0.296588,0.564436,0.806009,0.541994,1
35997,0.852159,0.212120,0.175924,0.730995,0.299303,0.337091,0.550591,0.095071,0.602569,0.918455,...,0.461448,0.687774,0.072004,0.730737,0.165870,0.774952,0.134323,0.089092,0.554625,1
35998,0.917705,0.441680,0.424490,0.691614,0.718346,0.032512,0.535217,0.764422,0.092839,0.050027,...,0.821959,0.723113,0.707863,0.309144,0.348293,0.913826,0.490889,0.545571,0.588099,0


## Validation Data

In [22]:
def make_batches(test_data): 
        data=np.array(test_data)
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches

In [23]:
def process_validation_data(test,encoder_pos_class,encoder_neg_class,batch_size=32):
   
    # From the test data ,separate positive and negative class data and remove the class column

    test_pos_class=test[test['output']==1]
    test_neg_class=test[test['output']==0]
    del test_pos_class['output']
    del test_neg_class['output']
       
       
    # This function make batches of the data stream
     
    def make_batches(test_data):
        data=np.array(test_data)
        batch_size=32
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches
    #
    
    batches_pos=make_batches(test_pos_class)
    batches_neg=make_batches(test_neg_class)
    
    # This function computes reconconstruction error for each instance as well as average reco.error of a batch passed to it. 
    def compute_instance_loss_batch_loss(batch,batch_size,encoder):
        mse_list=[]
        mse_sum=0  
        for i in range(0,batch.shape[0]):
            ROW = np.array([batch[i]])
            pred= encoder.predict(ROW) 
            mse = np.mean(np.power(batch[i] - pred, 2))
            mse_list.append(mse)
            mse_sum+=mse
        avg_mse=mse_sum/batch_size
        return mse_list,avg_mse   

            
    # This function computes recon.errr of all the batches .  
    def process_all_batches(batches,encoder):
        batch_avg_mse=[]
        batch_mse_values={}
        for b in batches:
            print(b,end=" ")
            mse_list,average_mse=compute_instance_loss_batch_loss(batches[b],batch_size,encoder)
            batch_avg_mse.append(average_mse)
            batch_mse_values[b]=mse_list
        return batch_avg_mse,batch_mse_values

    batch_avg_mse_pos,batch_mse_values_pos=process_all_batches(batches_pos,encoder_pos_class)
    batch_avg_mse_neg,batch_mse_values_neg=process_all_batches(batches_neg,encoder_neg_class)
       
    
    
    return  batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg
    
    
    
   
    

In [24]:
batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg=process_validation_data(test,encoder_pos_class,encoder_neg_class)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 

In [25]:
k_parameter=list(np.arange(1.0, 4.5, 0.5))
alpha_parameter=['median','max']
logical_parameter=['and','or']
import itertools

hyperparameter_space = {
    'k': k_parameter,
    'count_threshold_measure':alpha_parameter,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))
all_combinations

[(1.0, 'median', 'and'),
 (1.0, 'median', 'or'),
 (1.0, 'max', 'and'),
 (1.0, 'max', 'or'),
 (1.5, 'median', 'and'),
 (1.5, 'median', 'or'),
 (1.5, 'max', 'and'),
 (1.5, 'max', 'or'),
 (2.0, 'median', 'and'),
 (2.0, 'median', 'or'),
 (2.0, 'max', 'and'),
 (2.0, 'max', 'or'),
 (2.5, 'median', 'and'),
 (2.5, 'median', 'or'),
 (2.5, 'max', 'and'),
 (2.5, 'max', 'or'),
 (3.0, 'median', 'and'),
 (3.0, 'median', 'or'),
 (3.0, 'max', 'and'),
 (3.0, 'max', 'or'),
 (3.5, 'median', 'and'),
 (3.5, 'median', 'or'),
 (3.5, 'max', 'and'),
 (3.5, 'max', 'or'),
 (4.0, 'median', 'and'),
 (4.0, 'median', 'or'),
 (4.0, 'max', 'and'),
 (4.0, 'max', 'or')]

In [26]:
len(all_combinations)

28

### AEDDM Framwework

In [27]:
def detect_at_batch_level(test_batch,b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list):
    
    # Layer 1 Variables 
    mse_list_layer1=[]     # Holds the recon loss values predicted by Layer 1 Autoencoder for a single batch
    exceed_count_layer1=0  # How many instances exceed layer one instance level threshold
    mse_sum=0              # sum of recon.error values from layer 1 AE for this batch
    
    
    
    # Layer 2 Variables 
    mse_list_layer2=[]      # Holds the recon.error values predicted by Layer 2 Autoencoder for a single batch
    exceed_count_layer2=-1  # If a batch is not passed to the layer2 AE , then exceed count value will be -1 . This is symbolic.
    mse_sum_layer2=0        # sum of recon.error values from layer 2 AE for this batch
    
        
    layer1_excede_list=[] # Holds the batch numbers of bathces exceeding layer1 thresholds
    #all_excede_list=[]    # Hold the batch number of batches exceeding both Layer-1 and Layer-2 AE Thresholds 
    layer_one_instance_exceed_list=[] # Holds the indices of instances exceeding layer 1 instance threshold
    layer_two_instance_exceed_list=[] # Holds the indices of instances exceeding layer 2 instance thresholds
     
      
    # Determine Layer 1 and Layer AE and their associated thresholds
    layer_one_batch_thres= batch_thres_pos if batch_thres_pos<batch_thres_neg  else batch_thres_neg
    layer_two_batch_thres= batch_thres_pos if batch_thres_pos>batch_thres_neg  else batch_thres_neg
    layer_one_encoder= encoder_pos_class if batch_thres_pos<batch_thres_neg else encoder_neg_class
    layer_two_encoder= encoder_pos_class if batch_thres_pos>batch_thres_neg else encoder_neg_class
    
    layer_one_count_threshold=count_thresh_pos if batch_thres_pos<batch_thres_neg  else count_thres_neg
    layer_two_count_threshold=count_thresh_pos if batch_thres_pos>batch_thres_neg  else count_thres_neg
    
    layer1_ins_thresh=instance_thresh_pos if  batch_thres_pos<batch_thres_neg  else instance_thresh_neg
    layer2_ins_thresh=instance_thresh_pos if  batch_thres_pos>batch_thres_neg  else instance_thresh_neg
    
    # Pass each instance of a batch to Layer 1 AE. Compute Batch MSE and Number of Instances excceding Layer 1 Instance Thresh       
    avg_mse_layer1=0
    for i in range(0,test_batch.shape[0]):
        ROW = np.array([test_batch[i]])
        pred= layer_one_encoder.predict(ROW) 
        mse = np.mean(np.power(test_batch[i] - pred, 2))
        mse_list_layer1.append(mse)
        if mse>layer1_ins_thresh:
            exceed_count_layer1+=1
            layer_one_instance_exceed_list.append(i)
        mse_sum+=mse    
    avg_mse_layer1=(mse_sum)/len(test_batch)
    avg_mse_layer2=0 
    # Check if This batch exceeds both Layer 1 batch and count thresholds
    if ((avg_mse_layer1>layer_one_batch_thres) and ( exceed_count_layer1 >layer_one_count_threshold) ):
        layer1_excede_list.append(b)# Keep track of batches exceesding layer 1 thresholds
        exceed_count_layer2=0
        # Pass each instance of this batch to Layer 2 AE. Compute Batch MSE and Number of Instances excceding Layer 2 Instance Thresh
        for i in range(0,test_batch.shape[0]):
            ROW = np.array([test_batch[i]])
            pred= layer_two_encoder.predict(ROW) 
            mse = np.mean(np.power(test_batch[i] - pred, 2))
            mse_list_layer2.append(mse)
            if mse>layer2_ins_thresh:
                exceed_count_layer2+=1
                layer_two_instance_exceed_list.append(i)
            mse_sum_layer2+=mse    
        avg_mse_layer2=(mse_sum_layer2)/len(test_batch)
        
        if  (avg_mse_layer2 > layer_two_batch_thres) and (exceed_count_layer2>layer_two_count_threshold):
            all_excede_list.append(b)# Keep track of batches exceesding layer 2 thresholds
          
    return  all_excede_list,mse_list_layer1 , exceed_count_layer1 ,avg_mse_layer2,exceed_count_layer2,len(layer_two_instance_exceed_list),layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list,layer_two_instance_exceed_list,avg_mse_layer1 

In [28]:
def detect_stream_drift(batches,encoder_pos_class,encoder_neg_class,batch_thres_pos,batch_thres_neg, count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg,logical):
    exceed_count_layer2_instance_thresh={} # Holds Number of instances exceeding thresholds of Both Layer 1 and layer 2 AE.
    
    mse_dict_L1={}         # Holds batchwise recon.error values from Layer 1 AE
    exceed_count_L1={}     # Batchwise number of Instances exceeding layer 1 count threshold
    layer_one_instance_exceed_list={} # batch wise list of instances ( indices ) exceeding Layer 1 count threshold
    avg_mse_l1={}
    
    mse_dict_L2={}        # Holds batchwise recon. error  values from Layer 2 AE
    exceed_count_L2={}    # Batchwise number of Instances exceeding layer 2 count threshold
    layer_two_instance_exceed_list={} # For each batch maintains the indices where recon.error exceeds layer 2 threshold
    avg_mse_l2={}         #
    all_excede_list=[]
    n=0
    for b in batches:
        print("\n\n")
        print("**************")
        print('\nBatch Number : {}'.format(b))
        all_excede_list,mse_dict_L1[b],exceed_count_L1[b],avg_mse_l2[b],exceed_count_L2[b],exceed_count_layer2_instance_thresh[b],layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list[b],layer_two_instance_exceed_list[b],avg_mse_l1[b]= detect_at_batch_level(batches[b],b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list)
             
        
        print('\nData Points Exceeding Layer 1 Encoder Instance Threshold : {} '.format(layer_one_instance_exceed_list[b]))
        print('\nData Points Exceeding Layer 2 Encoder Instance Threshold: {}'.format(layer_two_instance_exceed_list[b])) 
        print('\nNumber of Data Points Exceeding Layer 2 Encoder Instance thresholds: {}'.format(exceed_count_layer2_instance_thresh[b]))
        
    mse_list_layer1=return_list_of_dict_values(avg_mse_l1)
    exceed_count_list_layer1=return_list_of_dict_values(exceed_count_L1)
    
    
    ## Detect Drift at Batch Level 
    print ("\n Drift Detection at Batch Level\n")
    exceed_list=return_list_of_dict_values(exceed_count_L2)
    mse_list=return_list_of_dict_values(avg_mse_l2)
    score,col_values=detect_drift(mse_list,exceed_list,layer_two_batch_thres,layer_two_count_threshold,logical)
    
    
    return score,col_values,all_excede_list,exceed_count_layer2_instance_thresh ,exceed_count_L2,avg_mse_l2 ,mse_list_layer1,exceed_count_list_layer1




In [29]:
def compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=20
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if(eval(f"({batch_mse[i]} > {Thresh}) {logical} ({exceed_list[i]} > {count_thresh})")):
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point)
    score=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={score}")
    
    col_values=[WnDs,DnDs,delay,WdDs,DdDs,score]
    
    return score,col_values


In [30]:
results_df_p=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [31]:
# This function computes how many instances in a batch exceed instance threshold
def threshold_exceed_count(batch_mse_values, thr):
    exceed_count={}
    for key in batch_mse_values.keys():
        count=0
        list=batch_mse_values[key]
        for a in range(0,len(list)):
            if list[a]>thr:
                count+=1
        exceed_count[key]=count 
    values = exceed_count.values()
    
    return exceed_count

def compute_count_threshold(exceed_count_dict,alpha):
    exceed_list=[]
    for key in exceed_count_dict.keys():
        exceed_list.append(exceed_count_dict[key]) 
    #print(" from with in count threshold")
    #print(alpha)
    method = getattr(np,alpha)  # this enables np.median() or np.max() based on alpha value
    count_threshold=method(exceed_list)    
    return count_threshold
   
def compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse,batch_avg_mse_values,k,alpha,N=100):
        # Computes Instance Threshold
        zscore_list=[]
        for i in range(0,N):
            value_list=batch_avg_mse_values[i]

            mean=np.mean(value_list)
            sigma=np.std(value_list)
            thres_zscore=mean+k*sigma
            zscore_list.append(thres_zscore)
        # Computes batch threshold
        mean_batch=np.mean(batch_avg_mse)
        sigma_batch=np.std(batch_avg_mse)    
        batch_thresh=mean_batch+k*sigma_batch
        
        method = getattr(np,alpha)
        instance_thresh = method(zscore_list)
        
        return  instance_thresh, zscore_list , batch_thresh


    
# Make batches of the normal data stream     
s=stream.copy()
del s['output']
batches_n=make_batches(s)
#print(batches_n)
# For testing purpose choose 20 batches only
batches_n=dict(itertools.islice(batches_n.items(), 20))
    
# Drifted Data Stream
drifted=drifted_data[0:1280]
batches_d=make_batches(drifted)    
    
    
    
def return_list_of_dict_values(d):
    values_list=[]
    for key in d.keys():
        values_list.append(d[key])
    return values_list
    

    
    
def pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg):
    # Define the parametric space
    k_parameter=list(np.arange(0.5, 4.5, 0.5))
    alpha_parameter=['median','max']
    logical_parameter=['and','or']
    results={}
   
    global results_df_p
    row=0
    
    
    for k in k_parameter:
        for alpha in alpha_parameter:
            for logical in logical_parameter:
                print("********************************")
                
                print(k,alpha,logical)
                values=[k, alpha, logical]
                parameters=(k,alpha,logical)
                instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
                instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
                #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
                # Counts the MSE values exceeding threshold in each batch 
                exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
                exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
                count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
                count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
                print("Thresholds for Positive AE")
                #print(alpha)
                print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
                print("Thresholds for Nagative AE")
                print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
                # AEDDM Starts Here 
                
                thres_zscore_batch_pos=batch_thresh_pos
                thres_zscore_batch_neg=batch_thresh_neg
                             
                
                #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
                    #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
                print(" Drifted Data Processing Starts Here")
                
                score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    
        
                results[parameters]=score
                values.extend(col_values)
                results_df_p = results_df_p.append(pd.Series(values, index=results_df_p.columns), ignore_index=True)
                print(results_df_p)
    
    return results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d

In [32]:
results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d=pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg)

********************************
0.5 median and
Thresholds for Positive AE
Batch Thresold: 0.054948107798222255 , Count Threshold: 6.0
Thresholds for Nagative AE
Batch Thresold: 0.05846562055673665 , Count Threshold: 8.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 4, 6, 7, 8, 9, 11, 14, 15, 16, 21, 22, 24, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 5, 6, 8, 10, 11, 12, 13, 17, 18, 19, 20, 21, 23, 24, 25, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 22



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 9, 12, 13, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16, 22, 23, 24, 26, 27, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 20






Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 30



**************

Batch Number : 23

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 31



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4,


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 4, 6, 7, 8, 9, 11, 14, 15, 16, 21, 22, 24, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 5, 6, 8, 10, 11, 12, 13, 17, 18, 19, 20, 21, 23, 24, 25, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 22



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 9, 12, 13, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16, 22, 23, 24, 26, 27, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 20



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 4, 5, 8, 9, 11, 13, 17, 18, 23, 27, 28, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 5, 6, 7, 10, 12, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 29, 30, 31]

Nu


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 30



**************

Batch Number : 23

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 31



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4,


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 6, 7, 8, 9, 11, 15, 16, 21, 24, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [10, 17, 18, 25, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 9, 12, 13, 14, 17, 18, 19, 20, 24, 25, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [10, 11, 12, 24, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 6



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 4, 8, 9, 11, 13, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 3, 12, 20, 21, 22, 24, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 8



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 4, 8, 10, 11, 13, 15, 17, 19, 20, 23, 24, 25, 28]

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [17] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 6, 17]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [14] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [14]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [11, 24, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 28

Data Points Exceeding Layer 1 Enc


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 7

Data Points Exceeding Layer 1 Encoder Instance Threshold : [18] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [4]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 8

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 9

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 10

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 4

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 



Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 36

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 37

Data Points Exceeding Layer 1 Encoder Instance Threshold : [24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 38

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 39

Data Points Exceeding Layer 1 Encoder Instance Threshold 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 22

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 23

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : 

In [33]:
results_df_p

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.5,median,and,20,20,0,18,18,114
1,0.5,median,or,20,20,0,18,18,114
2,0.5,max,and,1,1,0,18,18,171
3,0.5,max,or,20,20,0,18,18,114
4,1.0,median,and,20,20,0,18,18,114
5,1.0,median,or,20,20,0,18,18,114
6,1.0,max,and,0,0,20,0,0,60
7,1.0,max,or,0,0,0,18,18,174
8,1.5,median,and,17,17,0,18,18,123
9,1.5,median,or,20,20,0,18,18,114


In [34]:
results_df=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [35]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Categorical, Real


def aeddm_function(x):
    global results_df
    row=0
    k,alpha,logical=x
    values=[k, alpha, logical]
    #results_df.iloc[row :3] = values
        
    print("***************************************************")
    print(k,alpha,logical)
    parameters=(k,alpha,logical)
    instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
    instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
    #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
    # Counts the MSE values exceeding threshold in each batch 
    exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
    exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
    count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
    count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
    print("Thresholds for Positive AE")
    #print(alpha)
    print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
    print("Thresholds for Nagative AE")
    print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
    # AEDDM Starts Here 
                
    thres_zscore_batch_pos=batch_thresh_pos
    thres_zscore_batch_neg=batch_thresh_neg
                             
                
    #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
        #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
             #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    print(" Drifted Data Processing Starts Here")
                
    score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    #value+=100
    print(col_values)
    print(values)
    values.extend(col_values)
    print(values)
    
    results_df = results_df.append(pd.Series(values, index=results_df.columns), ignore_index=True)
    print(results_df)
    
    values=[]
    col_values=[]
    print(k,alpha,logical)
    return score 



In [36]:
# Define the search space
row=0
search_space = [
    Real(0.5, 4),  # Numerical parameter k with range 1 to 4
    Categorical(['median', 'max']),  # Categorical parameter alpha
    Categorical(['and', 'or'])  # Categorical parameter logical
]

# Perform the Bayesian optimization (maximization)
result = gp_minimize(
    #aeddm_function,
    lambda x: -aeddm_function(x),  # Negate the function to maximize
    dimensions=search_space,
    n_calls=20,  # Number of iterations
    acq_func='EI',  # Expected Improvement for maximization
    random_state=1  # Set a random seed for reproducibility
)

# Get the best parameters found
best_params = result.x
print("Best parameters:", best_params)

***************************************************
3.9901468382860408 median and
Thresholds for Positive AE
Batch Thresold: 0.0800411080952613 , Count Threshold: 0.0
Thresholds for Nagative AE
Batch Thresold: 0.0794252323940236 , Count Threshold: 0.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [12, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [24]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0




Data Points Exceeding Layer 1 Encoder Instance Threshold : [21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 4, 5, 24, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 35

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 36

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 37

Data Points Exceeding Layer 1 Encoder Instance Threshold : [16, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [16, 24]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 2



**************

Batch Number : 38

Data Points Exceeding Layer 1 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 23

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 5, 7, 13, 20, 22, 23, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 8, 18, 20, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 13

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 21, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [9, 18, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 14

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 5, 6, 8, 9, 10, 15, 17, 18, 22, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 12, 15, 19, 24, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 6



**************

Batch Number : 15

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 4, 7, 9, 11, 15, 22, 25, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 8, 13, 17


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 4, 5, 7, 9, 11, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 7, 8, 10, 11, 16, 17, 20, 21, 22, 24, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 13



**************

Batch Number : 38

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 5, 6, 8, 9, 11, 13, 14, 17, 18, 19, 20, 21, 22, 26, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 4, 5, 11, 12, 14, 15, 19, 21, 22, 25, 26, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14



**************

Batch Number : 39

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 4, 5, 7, 9, 11, 13, 14, 15, 19, 23, 24, 25, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 3, 11, 12, 13, 14, 15, 18, 20, 21, 22, 23, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thr

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 4, 8, 11, 15, 18, 19, 21, 23, 24, 25, 26, 27, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 3, 4, 6, 7, 8, 10, 16, 17, 19, 20, 22, 28, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14



**************

Batch Number : 7

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 4, 5, 10, 15, 16, 18, 22, 24, 25, 26, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 4, 6, 7, 8, 9, 11, 14, 16, 17, 18, 20, 21, 24, 27, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 8

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 6, 8, 11, 13, 14, 16, 21, 22, 24, 27, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 17, 18, 19, 20, 21, 22, 23, 24, 25, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 21



*******


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 28



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 29



**************

Batch Number : 30

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1

***************************************************
0.5 median and
Thresholds for Positive AE
Batch Thresold: 0.054948107798222255 , Count Threshold: 6.0
Thresholds for Nagative AE
Batch Thresold: 0.05846562055673665 , Count Threshold: 8.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 4, 6, 7, 8, 9, 11, 14, 15, 16, 21, 22, 24, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 5, 6, 8, 10, 11, 12, 13, 17, 18, 19, 20, 21, 23, 24, 25, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 22



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 9, 12, 13, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16, 22, 23, 24, 26, 27, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 30



**************

Batch Number : 23

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 31



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4,

Best parameters: [1.4556749097586161, 'max', 'or']


In [37]:
results_df_EI=results_df.copy()

In [38]:
results_df_EI

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,3.990147,median,and,0,0,3,1,1,114
1,3.996642,max,and,0,0,20,0,0,60
2,1.857688,median,or,17,17,0,18,18,123
3,3.462088,max,or,0,0,20,0,0,60
4,2.052085,max,or,0,0,13,1,1,84
5,3.698867,max,and,0,0,20,0,0,60
6,3.786947,median,or,1,1,3,2,2,114
7,3.309651,max,or,0,0,20,0,0,60
8,3.527571,median,or,1,1,3,2,2,114
9,1.455675,max,or,0,0,0,18,18,174


In [39]:
results_df=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [40]:
# Define the search space
search_space = [
    Real(0.5, 4),  # Numerical parameter k with range 1 to 4
    Categorical(['median', 'max']),  # Categorical parameter alpha
    Categorical(['and', 'or'])  # Categorical parameter logical
]

# Perform the Bayesian optimization (maximization)
result = gp_minimize(
    
    lambda x: -aeddm_function(x),  # Negate the function to maximize
    dimensions=search_space,
    n_calls=20,  # Number of iterations
    acq_func='PI',  # Expected Improvement for maximization
    random_state=1  # Set a random seed for reproducibility
)

# Get the best parameters found
best_params = result.x
print("Best parameters:", best_params)

***************************************************
3.9901468382860408 median and
Thresholds for Positive AE
Batch Thresold: 0.0800411080952613 , Count Threshold: 0.0
Thresholds for Nagative AE
Batch Thresold: 0.0794252323940236 , Count Threshold: 0.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [12, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [24]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0




Data Points Exceeding Layer 1 Encoder Instance Threshold : [21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 4, 5, 24, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 35

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 36

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 37

Data Points Exceeding Layer 1 Encoder Instance Threshold : [16, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [16, 24]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 2



**************

Batch Number : 38

Data Points Exceeding Layer 1 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 23

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 5, 7, 13, 20, 22, 23, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 8, 18, 20, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 13

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 21, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [9, 18, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 14

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 5, 6, 8, 9, 10, 15, 17, 18, 22, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 12, 15, 19, 24, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 6



**************

Batch Number : 15

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 4, 7, 9, 11, 15, 22, 25, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 8, 13, 17


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 4, 5, 7, 9, 11, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 7, 8, 10, 11, 16, 17, 20, 21, 22, 24, 29]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 13



**************

Batch Number : 38

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 5, 6, 8, 9, 11, 13, 14, 17, 18, 19, 20, 21, 22, 26, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 4, 5, 11, 12, 14, 15, 19, 21, 22, 25, 26, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 14



**************

Batch Number : 39

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 4, 5, 7, 9, 11, 13, 14, 15, 19, 23, 24, 25, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 3, 11, 12, 13, 14, 15, 18, 20, 21, 22, 23, 26]

Number of Data Points Exceeding Layer 2 Encoder Instance thr

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 6, 21, 22] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 9

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 10

Data Points Exceeding Layer 1 Encoder Instance Threshold : [13, 16, 21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 11

Data Points Exceeding Layer 1 Encoder Instance Threshold : [9, 16, 21, 25] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 12

Data Points Exceeding Lay

***************************************************
1.4255216623938336 max or
Thresholds for Positive AE
Batch Thresold: 0.06160230214173863 , Count Threshold: 3
Thresholds for Nagative AE
Batch Thresold: 0.06402371759092954 , Count Threshold: 2
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 8] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [12, 20, 24] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 10, 11, 13, 15, 18, 20, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 33

Data Points Exceeding Layer 1 Encoder Instance Threshold : [14, 20, 22, 25, 26, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [9, 25, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 34

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 4, 5, 9, 24, 26, 27, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [21]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 35

Data Points Exceeding Layer 1 Encoder Instance Threshold : [9, 14, 18, 19, 22, 26, 28, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Enc


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 6, 21, 22] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 9

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 10

Data Points Exceeding Layer 1 Encoder Instance Threshold : [13, 16, 21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 11

Data Points Exceeding Layer 1 Encoder Instance Threshold : [9, 16, 21, 25] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 12

Data Points Exceeding Lay

Best parameters: [1.4556749097586161, 'max', 'or']


In [41]:
max_value = result.fun  # Access the maximum function value
print("Maximum value:", max_value)
print("Best parameters:", result.x)

Maximum value: -174
Best parameters: [1.4556749097586161, 'max', 'or']


In [42]:
results_df

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,3.990147,median,and,0,0,3,1,1,114
1,3.996642,max,and,0,0,20,0,0,60
2,1.857688,median,or,17,17,0,18,18,123
3,3.462088,max,or,0,0,20,0,0,60
4,2.052085,max,or,0,0,13,1,1,84
5,3.698867,max,and,0,0,20,0,0,60
6,3.786947,median,or,1,1,3,2,2,114
7,3.309651,max,or,0,0,20,0,0,60
8,3.527571,median,or,1,1,3,2,2,114
9,1.455675,max,or,0,0,0,18,18,174


In [43]:
Thresh=0.212964
count_thresh=0
logical='and'

In [44]:
drift_point=20
WnDs= 0 
WdDs=0 
DnDs=0          
delay=None     
DdDs=0   

In [45]:
def compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift2(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=20
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if(eval(f"({batch_mse[i]} > {Thresh}) {logical} ({exceed_list[i]} > {count_thresh})")):
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point)
    value=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={value}")
    
    

In [46]:
detect_drift2(avg_mse_l2_list_d,exceed_count_L2_countThresh_d, Thresh,count_thresh,logical)

Number of Drifted Batches0
All Drifted Batches:  []
Drifted Batches before Drift Point:  []
Drifted Batches after Drift Point:  []
WnDs=0 ,WdDs=0 , DnDs=0,DdDs=0 , delay= 20 , value=60
